# 고급 컨볼루션 신경망

## tf.keras 추정기 모델로 패션 MNIST 분류

In [1]:
import os
import time
import tensorflow as tf
import numpy as np

# 데이터 불러오기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
train_images.shape,train_labels.shape,test_images.shape,test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [2]:
# 상수
LABEL_DIMENSIONS = 10  # 예측하려는 범주의 수 (0~9)
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

## 정규화 + 이미지 변환 + 채널추가
# 훈련이미지
train_images = np.asarray(train_images, dtype=np.float32) / 255
train_images = train_images.reshape((TRAINING_SIZE, 28, 28, 1))

# 테스트 이미지
test_images = np.asarray(test_images, dtype=np.float32) / 255
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))

# 레이블 데이터 원핫인코딩 + dtype=float32로 변환
train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS).astype(np.float32)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS).astype(np.float32)

print(train_images.dtype,train_labels.dtype)
print(train_images.shape,train_labels.shape)
print(test_images.dtype,test_labels.dtype)
print(test_images.shape,test_labels.shape)

float32 float32
(60000, 28, 28, 1) (60000, 10)
float32 float32
(10000, 28, 28, 1) (10000, 10)


In [18]:
## tf.keras 함수형 API로 컨볼루션 모델 구축
# model = add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(inputs))) 와 동일
inputs = tf.keras.Input(shape=(28,28,1))  
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(LABEL_DIMENSIONS, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 256)         147712    
_________________________________________________________________
flatten_3 (Flatten)          (None, 2304)              0   

In [19]:
# 컴파일
# from tensorflow.keras import optimizers 를 하지 않았을때는 tf 자체에서 호출해야함
optimizer = tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model

In [20]:
# 전략을 정의 - CPU = None, GPU = tf.distribute.MirroredStrategy()
strategy = None
#strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)
config

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


In [21]:
# tf.keras 모델을 편리한 추정기로 변환
estimator = tf.keras.estimator.model_to_estimator(model, config=config)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/fs/qjp88bt91vn_8kbs8qkcc_z00000gn/T/tmphynptkmf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7ffe60b5cc10>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_nu

In [7]:
# 훈련과 테스트용 입력 함수를 정의
def input_fn(images, labels, epochs, batch_size):
    # 입력을 데이터셋으로 변환
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    # 예시를 섞고, 반복하고, 배치한다.
    SHUFFLE_SIZE = 5000
    dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
    dataset = dataset.prefetch(None)
    
    # 반환
    return dataset
print(input_fn)

<function input_fn at 0x7f9c13f5a0e0>


In [23]:
## 훈련
# 상수
BATCH_SIZE = 512
EPOCHS = 50
estimator_train_result = estimator.train(input_fn=lambda:input_fn(train_images, train_labels,
                                                                 epochs=EPOCHS,
                                                                 batch_size=BATCH_SIZE))
print(estimator_train_result)

INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcas

In [24]:
estimator.evaluate(lambda:input_fn(test_images, 
                                   test_labels,
                                   epochs=1,
                                   batch_size=BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-19T10:17:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/fs/qjp88bt91vn_8kbs8qkcc_z00000gn/T/tmphynptkmf/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.17501s
INFO:tensorflow:Finished evaluation at 2020-06-19-10:17:39
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.8146, global_step = 5860, loss = 0.48601514
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /var/folders/fs/qjp88bt91vn_8kbs8qkcc_z00000gn/T/tmphynptkmf/model.ckpt-5860


{'accuracy': 0.8146, 'loss': 0.48601514, 'global_step': 5860}

## 전이학습용 심층 인셉션-v3 망
- 구글에서 만들어 제공하는 아주 깊은 컨볼루션 신경망
- 새로운 작업에 적절하게 미세조정하여 사용 가능하다.
- 모델의 기본입력 크기 : 3채널 299 x 299

In [13]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
# 사전에 훈련된 기본 모델 생성
base_model = InceptionV3(weights='imagenet', include_top=False) # include_top : 마지막 3개의 계층이 제거됨

87916544/87910968 [==============================] - 8s 0us/step


In [20]:
# outputs = 1000개
#base = InceptionV3(weights='imagenet', include_top=True)
#base.summary()

96116736/96112376 [==============================] - 51s 1us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 149, 149, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 149, 149, 32) 0           batch_normalization_94[0][0]     
_____________________

In [17]:
x = base_model.output
# 완전 연결 계층을 첫 번째 계층으로 추가한다.
x = layers.Dense(1024, activation='relu')(x)
# 200개 부류를 가진 로지스틱 계층을 마지막 계층으로 한다.
predictions = layers.Dense(200, activation='softmax')(x)
# 훈련할 모델
model = models.Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [16]:
# 모든 컨볼루션 인셉션 V3 계층을 고정
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# 모델 컴파일(계층을 비훈련으로 설정한 다음 수행해야 한다)
model.compile(optimizer='rmsporp', loss='categorical_crossentropy', metrics=['accuracy'])

# 몇 개의 에폭 동안 새로운 데이터로 모델 훈련
model.fit_generator(...)

- 그런 다음 인셉션의 최상위 계층을 고정하고, 일부 인셉션 계층을 미세 조정한다.
- 이 예제에서는 처음 172개의 계층을 고정하기로 결정함

In [19]:
# 상위 2개 인셉션 블록을 훈련하기로 결정
# 즉, 처음 172개 블록을 고정하고, 나머지를 조정
for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True

In [ ]:
# 모델 최적화를 위해 미세조정하고자 재 컴파일
# 효과적으로 변경하려면 모델을 재 컴파일해야 한다.
# SGD를 낮은 학습률로 사용
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=.0001, momentum=.9), loss='categorical_crossentropy')

# 모델을 다시 한 번 훈련(이번에는 상위 2개 블록을 미세 조정)
# 상위 밀집 계층과 함께
model.fit_generator(...)

## 텐서플로 Hub
- 이 예제에서는 MobileNetV2를 사용한 간단한 이미지 분류기를 살펴봄
- 일단 Hub 계층을 래핑하는데 hub.KerasLayer()를 사용한다는것만 기억해두자

In [13]:
import matplotlib.pyplot as plt, tensorflow as tf, tensorflow_hub as hub,\
numpy as np, PIL.Image as Image

classifier_url = "http://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:'string'}
IMAGE_SHAPE = (224,224)
# tf.keras와 작업하고자 hub를 래핑
classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])
grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper = np.array(grace_hopper) / 255.0
result = classifier.predict(grace_hopper[np.newaxis, ...])
predicted_class = np.argmax(result[0], axis=-1)
print(predicted_class)

65536/61306 [================================] - 0s 0us/step
653


## 기타 CNN 아키텍쳐

### AlexNet

- 최초의 컨볼루션 망
- 단 8개의 계층으로 구성됨
- 처음5개는 최대풀링계층, 마지막 3개는 완전연결 계층

### ResNet
- 이전 계층을 더 깊은 계층으로 직접공급한다는 아이디어
- 소위 스킵 연결(skip connections) or 고속전방연결(fast-forword connections)
- 인간의 대뇌 피질층 VI뉴런이 중간계층을 건너뛰고 1계층에서 입력을 받는 점이 유사한 패턴을 가진다고 알려져있다.

### HighwayNets와 DenseNets
- 추가 가중치 행렬을 사용해 스킵 가중치를 학습, highwaynets
- 여러 개의 병렬 스킵이 있는 모델, densenets

### Xception(중요)
- 깊이별(deepwise)컨볼루션과 점별(pointwise)컨볼루션이라는 두 가지 기본 블록을 사용
- deepwise conv : 채널별(channel-wise) n x n 공간 컨볼루션
- pointwise conv : 1 x 1 컨볼루션
- Xception망은 인셉션 모듈의 극단적(extreme)버전 : 교차 채널 교정이라는 것을 위해 1x1 컨볼루션<br>
  을 사용해 각 출력 채널의 공간 교정을 개별적으로 매핑
- 인셉션에서 영감을 얻은 심층 컨볼루션 신경망 아키텍처
- Casing, HyperNets, DenseNets, Inception, Xception은 모두<br>
  tf.keras.applications와 TF-Hub에서 사전 훈련된 망 형태로 제공된다.

In [18]:
from tensorflow.keras.applications.xception import Xception
model = Xception(weights='imagenet', include_top=True)
model.summary()

91889664/91884032 [==============================] - 10s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
_________________________

In [19]:
model = Xception(weights='imagenet', include_top=False)
model.summary()

83689472/83683744 [==============================] - 8s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________

## 이미지에 대한 질문의 대답(VQA)
- 3 channel (224 x 224) for inputs

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models
# IMAGE
# 시각 처리를 위한 CNN 정의
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(224,224,3)))
cnn_model.add(layers.Conv2D(64, (3,3), activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
cnn_model.add(layers.Conv2D(128, (3,3), activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
cnn_model.add(layers.Conv2D(256, (3,3), activation='relu'))
cnn_model.add(layers.Conv2D(256, (3,3), activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Flatten())
cnn_model.summary()

# 적절한 입력으로 visual_model 정의
image_input = layers.Input(shape=(224,224,3))
visual_model = cnn_model(image_input)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 222, 222, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 111, 111, 128)     73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 109, 109, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 54, 54, 256)      

- 텍스트는 RNN으로 인코딩 할 수 있다.(지금은 그저 텍스트의 일부[질문]를 입력으로 받아 해당 텍스트의 특징 벡터를 생성하는 블랙박스 정도로 생각하자)

In [22]:
# 텍스트
# 텍스트 처리를 위한 RNN 모델 정의
question_input = layers.Input(shape=(100,), dtype='int32')
embedding = layers.Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
encoded_question = layers.LSTM(256)(embedding)

- 그런 다음 두 특징 벡터(하나는 이미지, 하나는 텍스트)는 하나의 결합 벡터로 병합돼 밀집 망의 입력으로 제공돼서 병합된 망을 생성한다.

In [26]:
# 인코딩된 질문과 시각 모델을 병합
merged = layers.concatenate([encoded_question, visual_model])
# 마지막에 밀집 망을 부가
output = layers.Dense(1000, activation='softmax')(merged)

# 병합된 모델 얻기
vqa_model = models.Model(inputs=[image_input, question_input], outputs=output)
vqa_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 256)     2560000     input_5[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 256)          525312      embedding[0][0]                  
______________________________________________________________________________________________

## 스타일 변환
- 소스 콘텐츠 이미지 p와 소스 스타일 이미지 a의 스타일을 공유하는 인공 이미지 x를 생성하는 작업
- 따라서 두가지 거리 함수가 필요하다(1.두 이미지의 내용이 얼마나 다른지 측정하는 거리함수, 2.두 이미지으 ㅣ스타일이 얼마나 다른지 측정하는 거리함수)
- 멋진 이미지를 생성하려면 생성된 이미지의 콘텐츠가 입력 이미지의 콘텐츠와 비슷해야(즉, 작은거리)한다.
- 따라서 거리는 표준 역전파로 최소화한다.

In [28]:
# 콘텐츠 거리
def get_content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - taeget))

# 스타일 거리
def gram_matrix(input_tensor):
    # 이미지 채널을 먼저 한다.
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1,channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

In [1]:
import numpy as np, tensorflow as tf
from IPython.display import clear_output
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image

In [2]:
# 이미지를 다운로드하고 Numpy 배열로 읽는다.
def download(url):
    name = url.split("/")[-1]
    image_path = tf.keras.utils.get_file(name, origin=url)
    img = image.load_img(image_path)
    return image.img_to_array(img)

# 픽셀을 (-1.0 ~ 1.0) 사이의 값으로 조정
def preprocess(img):
    return (img / 127.5) - 1

# 이전의 전처리를 취소
def deprocess(img):
    img = img.copy()
    img /= 2.
    img += .5
    img *= 255.
    return np.clip(img, 0, 255).astype('uint8')

# 이미지를 표시
def show(img):
    plt.figure(figsize=(12,12))
    plt.grid(False)
    plt.axis('off')
    plt.imshow(img)
# https://commons.wikimedia.org/wiki/File:Flickr_-_Nicolas_T_-_Big_Sky_(1).jpg
url = 'https://storage.googleapis.com/applied-dl/clouds.jpg'
img = preprocess(download(url))
#show(deprocess(img))

In [3]:
inception_v3 = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)

In [4]:
# 이 계층들의 활성화를 최대화한다.
names = ['mixed2','mixed3','mixed4','mixed5']
layers = [inception_v3.get_layer(name).output for name in names]

# 특징 추출 모델 생성
feat_extraction_model = tf.keras.Model(inputs=inception_v3.input, outputs=layers)

def forward(img):
    # 배치 생성
    img_batch = tf.expand_dims(img, axis=0)
    
    # 이미지를 인셉션을 통해 전방 전달, 앞서 선택한 계층의 활성화 함수 추출
    return feat_extraction_model(img_batch)

In [35]:
layers

[<tf.Tensor 'mixed2/Identity:0' shape=(None, None, None, 288) dtype=float32>,
 <tf.Tensor 'mixed3/Identity:0' shape=(None, None, None, 768) dtype=float32>,
 <tf.Tensor 'mixed4/Identity:0' shape=(None, None, None, 768) dtype=float32>,
 <tf.Tensor 'mixed5/Identity:0' shape=(None, None, None, 768) dtype=float32>]

In [37]:
feat_extraction_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, None, None, 3 96          conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization_8[0][0]      
____________________________________________________________________________________________

- 손실 함수는 고려한 모든 활성화 계층의 평균을 계층 자체의 유닛 개수로 정규화한 것이다.

In [5]:
def calc_loss(layer_activations):
    total_loss = 0
    
    for act in layer_activations:
        # 그래디언트 상승에서는, 이 값을 최대화하려 한다.
        # 따라서 이미지는 점진적으로 계층을 '자극'한다.
        loss = tf.math.reduce_mean(act)
        
        # 계층의 유닛 개수로 정규화
        loss = np.prod(act.shape)
        total_loss += loss
        
    return total_loss

- 이제 그래디언트 상승을 실행

In [7]:
# 훈련을 위해 이미지를 변수로 변환
img = tf.Variable(img)
steps = 400

for step in range(steps):
    with tf.GradientTape() as tape:
        activations = forward(img)
        loss = calc_loss(activations)
    
    gradients = tape.gradient(loss, img)
    # 그래디언트 정규화
    gradients /= gradients.numpy().std() + 1e-8
    
    # 그래디언트를 즉시 더해 이미지를 갱신
    img.assign_add(gradients)
    
    if step % 50 == 0:
        clear_output()
        print("Step %d, loss %f" %(step, loss))
        show(deprocess(img.numpy()))
        plt.show()
        
# 결과를 보자
clear_output()
show(deprocess(img.numpy()))

AttributeError: 'numpy.int64' object has no attribute '_id'

## CNN을 사용한 감정 분석

In [17]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing as pre
import tensorflow_datasets as tfds

max_len = 200
n_words = 10000
dim_embedding = 256
EPOCHS = 20
BATCH_SIZE = 500

def load_data():
    # 데이터 로드
    (X_train,y_train),(X_test,y_test) = datasets.imdb.load_data(num_words=n_words)
    # max_len으로 시퀀스 패딩
    X_train = pre.sequence.pad_sequences(X_train, maxlen=max_len)
    X_test = pre.sequence.pad_sequences(X_test, maxlen=max_len)
    return (X_train, y_train),(X_test, y_test)

- 적절한 CNN모델을 만든다. 여기서는 임베딩을 사용해 문서에서 일반적으로 관찰되는 희소 어휘를 din_embedding(256)차원의 밀집 특징공간에<br>
  매핑한다. 그런 다음 Conv1D를 사용하고 나서 평균화하고자 GlobalMaxPooling1D를 사용하고 두 개의 밀집 계층을 사용한다.<br>
  마지막 계층에는 이진 선택을 발화하는 단 하나의 뉴런(긍정 또는 부정)이 있다.

In [18]:
def build_model():
    model = models.Sequential()
    # 입력 - 임베딩 계층
    # 모델은 (batch, input_length) 크기의 정수 행렬을 입력으로 받는다.
    # 모델은 (input_length, dim_embedding) 차원의 출력을 가진다.
    # 입력에서 최대 정수는 n_words(어휘 크기)보다 작거나 같아야 한다.
    model.add(layers.Embedding(n_words, dim_embedding, input_length=max_len))
    
    model.add(layers.Dropout(.3))
    model.add(layers.Conv1D(256, 3, padding='valid', activation='relu'))
    
    # 각 n_words 특징에서 특징 벡터의 최댓값을 취한다.
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

(X_train,y_train),(X_test,y_test) = load_data()
X_train.shape,y_train.shape,X_test.shape,y_test.shape
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 256)          2560000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 256)          196864    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

- 이제 모델을 컴파일하고 Adam 최적화기와 이진 교차 엔트로피 손실 함수로 모델을 적합화한다.

In [15]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs/imdb_CNN')]

score = model.fit(X_train, y_train,
                 epochs=EPOCHS, batch_size=BATCH_SIZE,
                 validation_data=(X_test,y_test),
                 callbacks=callbacks)
score = model.evaluate(X_test,y_test, batch_size=BATCH_SIZE)
print("\nTest score:",score[0])
print("Test Accuracy:",score[1])

Epoch 1/20
50/50 [==============================] - 35s 699ms/step - loss: 0.6450 - accuracy: 0.6408 - val_loss: 0.5032 - val_accuracy: 0.7768
Epoch 2/20
50/50 [==============================] - 35s 707ms/step - loss: 0.3816 - accuracy: 0.8358 - val_loss: 0.2966 - val_accuracy: 0.8739
Epoch 3/20
50/50 [==============================] - 37s 732ms/step - loss: 0.2404 - accuracy: 0.9046 - val_loss: 0.2616 - val_accuracy: 0.8912
Epoch 4/20
50/50 [==============================] - 36s 723ms/step - loss: 0.1582 - accuracy: 0.9461 - val_loss: 0.2716 - val_accuracy: 0.8879
Epoch 5/20
50/50 [==============================] - 35s 708ms/step - loss: 0.0934 - accuracy: 0.9715 - val_loss: 0.3010 - val_accuracy: 0.8863
Epoch 6/20
50/50 [==============================] - 37s 738ms/step - loss: 0.0499 - accuracy: 0.9885 - val_loss: 0.3234 - val_accuracy: 0.8858
Epoch 7/20
50/50 [==============================] - 36s 726ms/step - loss: 0.0265 - accuracy: 0.9952 - val_loss: 0.3588 - val_accuracy: 0.8844

In [22]:
%load_ext tensorboard
#!rm -rf ./logs/
%tensorboard --logdir logs/imdb_CNN